In [40]:
import dask.dataframe as dd
import minio
import os
import json
import datetime
from dask_kubernetes.operator import KubeCluster

In [42]:
minio_client = minio.Minio(
    endpoint="digibuild.epu.ntua.gr:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)

In [43]:
cluster = KubeCluster(
    name="testcluster",
    env={"EXTRA_PIP_PACKAGES": "s3fs"} # add extra packages
)
cluster.add_worker_group(
    name="cagri",
    n_workers=2,
    resources={
        "requests":{"memory": "2Gi"},
        "limits": {"memory": "4Gi"}
    }
)

Output()

In [44]:
client = cluster.get_client()
client

/Users/panagiotiskapsalis/PycharmProjects/MARTEL-PROJECTS/DaskK8SExample/.venv/lib/python3.9/site-packages/distributed/client.py:1393: VersionMismatchWarning: Mismatched versions found

+---------+---------------+-----------------+-----------------+
| Package | Client        | Scheduler       | Workers         |
+---------+---------------+-----------------+-----------------+
| msgpack | 1.0.8         | 1.0.7           | 1.0.7           |
| python  | 3.9.6.final.0 | 3.10.12.final.0 | 3.10.12.final.0 |
| toolz   | 0.12.1        | 0.12.0          | 0.12.0          |
+---------+---------------+-----------------+-----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://10.1.5.53:8786' processes=2 threads=16, memory=8.00 GiB>

In [45]:
ddf = dd.read_csv(
    's3://prefect-data/nycflights.csv',
    storage_options={
        "client_kwargs": {
            "endpoint_url": 'http://digibuild.epu.ntua.gr:9000', 
            "aws_access_key_id":'minio', 
            "aws_secret_access_key":'minio123'
        }}
    )

In [46]:
ddf.columns

Index(['year', 'month', 'day', 'dep_time', 'dep_delay', 'arr_time',
       'arr_delay', 'carrier', 'tailnum', 'flight', 'origin', 'dest',
       'air_time', 'distance', 'hour', 'minute'],
      dtype='object')

In [47]:
ddf.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2013,6,30,940,15,1216,-4,VX,N626VA,407,JFK,LAX,313,2475,9,40
1,2013,5,7,1657,-3,2104,10,DL,N3760C,329,JFK,SJU,216,1598,16,57
2,2013,12,8,859,-1,1238,11,DL,N712TW,422,JFK,LAX,376,2475,8,59
3,2013,5,14,1841,-4,2122,-34,DL,N914DL,2391,JFK,TPA,135,1005,18,41
4,2013,7,21,1102,-3,1230,-8,9E,N823AY,3652,LGA,ORF,50,296,11,2


In [49]:
ddf.dep_delay.max().compute()

1301

In [50]:
len(ddf)

32735

In [52]:
ddf.groupby('origin').air_time.mean().compute()

origin
EWR    152.444567
JFK    178.266037
LGA    117.982120
Name: air_time, dtype: float64

In [53]:
ddf.groupby('origin').dep_delay.mean().compute()

origin
EWR    15.310339
JFK    12.265486
LGA    10.134896
Name: dep_delay, dtype: float64

In [36]:
client.get_versions(check=True)

/Users/panagiotiskapsalis/PycharmProjects/MARTEL-PROJECTS/DaskK8SExample/.venv/lib/python3.9/site-packages/distributed/client.py:4677: UserWarning: Mismatched versions found

+---------+---------------+-----------------+-----------------+
| Package | Client        | Scheduler       | Workers         |
+---------+---------------+-----------------+-----------------+
| msgpack | 1.0.8         | 1.0.7           | 1.0.7           |
| python  | 3.9.6.final.0 | 3.10.12.final.0 | 3.10.12.final.0 |
| toolz   | 0.12.1        | 0.12.0          | 0.12.0          |
+---------+---------------+-----------------+-----------------+
  warnings.warn(msg["warning"])


{'scheduler': {'host': {'python': '3.10.12.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '6.6.22-linuxkit',
   'machine': 'x86_64',
   'processor': 'x86_64',
   'byteorder': 'little',
   'LC_ALL': 'C.UTF-8',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.12.final.0',
   'dask': '2024.5.0',
   'distributed': '2024.5.0',
   'msgpack': '1.0.7',
   'cloudpickle': '3.0.0',
   'tornado': '6.4',
   'toolz': '0.12.0',
   'numpy': '1.26.4',
   'pandas': '2.2.2',
   'lz4': '4.3.3'}},
 'workers': {'tcp://10.1.5.43:39383': {'host': {'python': '3.10.12.final.0',
    'python-bits': 64,
    'OS': 'Linux',
    'OS-release': '6.6.22-linuxkit',
    'machine': 'x86_64',
    'processor': 'x86_64',
    'byteorder': 'little',
    'LC_ALL': 'C.UTF-8',
    'LANG': 'C.UTF-8'},
   'packages': {'python': '3.10.12.final.0',
    'dask': '2024.5.0',
    'distributed': '2024.5.0',
    'msgpack': '1.0.7',
    'cloudpickle': '3.0.0',
    'tornado': '6.4',
    'toolz': '0.12.0',
    'numpy':

In [37]:
cluster.close()

In [38]:
client.close()

In [14]:
os.path.join("data", "nycflights.csv")

'data/nycflights.csv'

In [ ]:
minio_client = 